# 模組 4.3: 目標編碼 (Target Encoding)

## 學習目標
- 理解目標編碼的基本原理及其強大的預測能力。
- 認識到目標編碼中固有的資料洩漏風險。
- 學習並實作一種穩健的、可以防止過擬合的目標編碼方法（使用交叉驗證）。
- 了解平滑 (Smoothing) 技術在目標編碼中的作用。

## 導論：利用目標變數的力量

目標編碼：「*能捕捉類別與目標之間的關係...但有過度擬合和數據洩漏的風險，需要非常謹慎地實施*」。

**目標編碼 (Target Encoding)**，也稱為均值編碼 (Mean Encoding)，是一種強大的監督式編碼方法。它直接利用目標變數的資訊來為類別特徵賦值。

**基本原理**: 將每個類別替換為該類別所對應的 **目標變數的平均值**。

這種方法能直接地、強力地捕捉到特徵與目標之間的關係，因此常常能在 Kaggle 等競賽中取得很好的效果。如果實施不當，它也是最容易導致 **數據洩漏** 和 **模型過度擬合** 的方法。

In [1]:
# 導入必要的函式庫
import pandas as pd
from sklearn.model_selection import KFold



## 1. 準備資料

In [2]:
# 我們使用一個簡單的資料集來清晰地展示原理
df = pd.DataFrame({
    'City': ['New York', 'London', 'London', 'New York', 'Paris', 'Paris', 'New York', 'London'],
    'Purchased': [1, 0, 0, 1, 1, 0, 1, 1] # 我們的目標變數
})

print("原始 DataFrame:")
display(df)



原始 DataFrame:


,City,Purchased
0,New York,1
1,London,0
2,London,0
3,New York,1
4,Paris,1
5,Paris,0
6,New York,1
7,London,1


## 2. 天真 (Naive) 的目標編碼：一個錯誤的示範

最直觀的方法是直接計算每個城市對應的 `Purchased` 均值，然後用它來編碼。讓我們看看這樣做為什麼是錯誤的。

In [3]:
# 計算每個城市的購買率均值
city_mean = df.groupby('City')['Purchased'].mean()
print("每個城市的平均購買率:")
print(city_mean)



每個城市的平均購買率:
City
London      0.333333
New York    1.000000
Paris       0.500000
Name: Purchased, dtype: float64


In [4]:
# 直接映射回原欄位
df['Naive_Target_Encoded'] = df['City'].map(city_mean)

print("\n天真的目標編碼結果:")
display(df)




天真的目標編碼結果:


,City,Purchased,Naive_Target_Encoded
0,New York,1,1.000000
1,London,0,0.333333
2,London,0,0.333333
3,New York,1,1.000000
4,Paris,1,0.500000
5,Paris,0,0.500000
6,New York,1,1.000000
7,London,1,0.333333


**問題在哪裡？—— 數據洩漏 (Data Leakage)**

當我們計算第 0 行 "New York" 的編碼值 (1.0) 時，我們使用了第 0 行自身的目標值 (1)。這意味著我們把 **答案 (目標值) 的一部分洩漏給了特徵**。模型在訓練時看到這個特徵，會輕易地學到「如果編碼值是 1.0，那麼目標很可能就是 1」，這會導致模型在訓練集上表現得非常好，但在從未見過的測試集上表現很差，因為測試集的目標值是未知的。這就是 **過度擬合**。

## 3. 穩健的目標編碼：使用 K-Fold 交叉驗證

為了防止上述的資料洩漏，我們必須確保在計算某一行 (sample) 的編碼值時，**不能使用該行自身的目標值**。一個穩健的方法是使用 K-Fold 交叉驗證策略。

**流程**:
1. 將訓練資料分成 K 折 (e.g., K=5)。
2. 對於每一折（比如第 1 折），我們稱之為 "驗證折" (validation fold)。
3. 使用 **除了這一折之外的所有其他折（K-1 折）** 的資料來計算目標均值。
4. 用計算出的均值來填充 "驗證折" 的編碼值。
5. 對所有 K 折重複此過程，這樣訓練集中的每一行都被賦予了一個在計算過程中沒有用到它自身目標值的編碼。
6. 最後，計算 **整個訓練集** 的目標均值，用來填充 **測試集**。


In [ ]:
# 重新開始，使用穩健的方法
df_robust = df[['City', 'Purchased']].copy()
df_robust['Robust_Target_Encoded'] = 0 # 初始化一個新欄位

# 設置 K-Fold
kf = KFold(n_splits=4, shuffle=True, random_state=42)

# 對每一折進行處理
for train_index, val_index in kf.split(df_robust):
    # 分出訓練折和驗證折
    train_fold = df_robust.iloc[train_index]
    val_fold = df_robust.iloc[val_index]
    
    # *** 只在訓練折上計算均值 ***
    means = train_fold.groupby('City')['Purchased'].mean()
    
    # *** 將均值應用到驗證折上 ***
    df_robust.loc[val_index, 'Robust_Target_Encoded'] = val_fold['City'].map(means)

# 處理在某些折中可能未出現的類別導致的 NaN
# 用全域均值填充
global_mean = df_robust['Purchased'].mean()
df_robust['Robust_Target_Encoded'].fillna(global_mean, inplace=True)
    
print("穩健的目標編碼結果:")
display(df_robust)



**觀察**:
注意看，現在 `New York` 的編碼值不再是單一的 1.0，而是根據它所在的驗證折，由其他資料計算得來。例如，第 0 行的編碼值 (0.666) 是在不包含它自身的情況下計算的，從而避免了直接的資料洩漏。

## 4. 平滑 (Smoothing)

對於出現次數很少的類別，僅用幾筆樣本計算出的目標均值可能非常不穩定（例如，某個城市只出現一次且用戶購買了，均值就是 1.0）。

**平滑** 是一種將 **局部均值** (該類別的均值) 與 **全域均值** (所有樣本的均值) 進行加權平均的技術，可以讓低頻類別的編碼值更可靠，更趨近於整體平均水平。

**公式**: `平滑編碼 = w * 局部均值 + (1 - w) * 全域均值`

其中 `w` 是一個權重，通常與該類別的樣本數量有關，樣本越多，`w` 越大，越相信局部均值。
`w = n / (n + m)`，`n` 是類別樣本數，`m` 是一個平滑參數。

## 總結

目標編碼是一種威力強大但風險極高的工具。

- **優點**:
  - 直接捕捉特徵與目標的關係，預測能力強。
  - 不增加特徵維度，適合處理高基數特徵。
- **缺點/風險**:
  - **極易過度擬合與資料洩漏**。
  - 對於低頻類別，編碼值不穩定。
- **核心要點**:
  - **絕對不能使用 "天真" 的方法**。
  - **必須使用基於交叉驗證的策略** 來防止資料洩漏。
  - 對於低頻類別，考慮使用 **平滑** 技術來增加編碼的穩健性。

由於其複雜性和風險，建議在對簡單編碼方法效果不滿意時，再謹慎地嘗試目標編碼。